In [12]:
import numpy as np, pandas as pd
import json, os, datetime, time
import talib as ta
import matplotlib.pyplot as plt, plotly
import requests
import MetaTrader5 as mt5
from beeprint import pp
import bios

def date_to_str(date):
    return date.strftime('%Y-%m-%d')

def date_to_ymd_str(date):
    # return list of yyy, mm, dd as string (2 digit)
    return str(date).split(' ')[0].split('-')

def strTZ_to_date(txt):
    # ex: txt = '2019-08-01T00:00:00.000Z'
    return datetime.datetime.strptime(txt,'%Y-%m-%dT%H:%M:%S.%fZ')


login =  1111
password =  "11111"
server =  "111-1111"

def connect(login=login, password=password, server=server):
    if not mt5.initialize(login=login, password=password,server=server):
        print("initialize() failed, error code =",mt5.last_error())

connect(login, password, server)

# storing infos functions

In [23]:
def update_json(assetType, broker, instrument, timeframe, symbol, columns, dic_resamp, symbol_infos):
    if f'{broker}.json' in os.listdir(f'D://Trading/Data/{assetType}'):
        with open(f'D://Trading/Data/{assetType}/{broker}.json','r') as f:
            dic = json.load(f)
            f.close()
    else:
        dic = {}
    if instrument not in dic:
        dic[instrument] = {}
    if timeframe not in dic[instrument]:
        dic[instrument][timeframe] = {}
    dic[instrument][timeframe][symbol] = {
        "columns": columns,
        "resampling_dict": dic_resamp,
        "infos": symbol_infos
    }
    with open(f'D://Trading/Data/{assetType}/{broker}.json','w') as f:
        json.dump(dic, f)
        f.close()

def update_infos(assetType, broker, instrument, timeframe, symbol, df):
    columns = [col for col in df.columns]    
    list_resamp_first_col = ['time', 'open', 'ask_open', 'bid_open']
    list_resamp_last_col = ['close', 'ask_close', 'bid_close']
    list_resamp_max_col = ['high', 'ask_high', 'bid_high']
    list_resamp_min_col = ['low', 'ask_low', 'bid_low']
    list_resamp_sum_col = ['size', 'foreignNotional', 'v_b', 'v_s', 'v_weigted_p', 'v_weigted_p_b', 'v_weigted_p_s', 'qty', 'qty_b', 'qty_s']
    list_resamp_mean_col = ['v_b_over_v', 'v_s_over_v', 'p_mean', 'v_weigted_p_over_p_mean', 'v_weigted_p_b_over_v_weigted_p', 'v_weigted_p_s_over_v_weigted_p', 'p_std', 'p_std_b', 'p_std_s', 'spread']
    dic_resamp = {}
    for col in df.columns:
        if col in list_resamp_first_col:
            dic_resamp[col] = 'first'
        elif col in list_resamp_last_col:
            dic_resamp[col] = 'last'
        elif col in list_resamp_max_col:
            dic_resamp[col] = 'max'
        elif col in list_resamp_min_col:
            dic_resamp[col] = 'min'
        elif col in list_resamp_sum_col:
            dic_resamp[col] = 'sum'
        elif col in list_resamp_mean_col:
            dic_resamp[col] = 'mean'
        else:
            dic_resamp[col] = 'mean'
            
    mt5_info = mt5.symbol_info(symbol)
    symbol_infos = {'point':mt5_info.point, 
                            'contract_value_in_usd':mt5_info.trade_contract_size, 
                            'volume_step':mt5_info.volume_step, 
                            'volume_max':mt5_info.volume_max, 
                            'min_point_stop':mt5_info.trade_stops_level,
                            'taker_fee':0,
                            'maker_fee':0,
                           }
    
    
    update_json(assetType, broker, instrument, timeframe, symbol, columns, dic_resamp, symbol_infos)

# get list_symbol

In [24]:
dic_assetType = {}
for symbol in mt5.symbols_get('*'):
    main_assetType = symbol.path.split('\\')[0]
    if main_assetType not in dic_assetType:
        dic_assetType[main_assetType] = 1
    else:
        dic_assetType[main_assetType] += 1
        
list_assetType = [k for k in dic_assetType.keys()]

for k,item in dic_assetType.items():
    print(f'{k} ({item})')

Stocks (174)
Forex (37)
Forex Major (9)
Crypto Currency (5)
CFDs (16)
Oil (3)
Commodities (9)
Nikkei (1)
Silver (2)
Gold (2)


## import monthly df

In [5]:
def get_df_month(symbol, year, month, add_date=False):
    init_date = datetime.datetime(year, month, 1)
    next_month = month % 12 + 1
    next_year = year + month // 12
    end_date = datetime.datetime(next_year, next_month, 1)
    rates = mt5.copy_rates_range(symbol, mt5.TIMEFRAME_M1, init_date, end_date)
    if rates is None:
        print(mt5.last_error())
        return pd.DataFrame()
    elif len(rates) > 0:
        df = pd.DataFrame(rates)
        df = df.drop('real_volume',axis=1)
        if add_date:
            df.loc[:,'date'] = pd.to_datetime(df.time, unit='s')
        for col in ['open', 'high', 'low', 'close']:
            df.loc[:,f'ask_{col}'] = df[f'{col}'] + df.spread * mt5.symbol_info(symbol).point
        df = df.rename(columns={'open':'bid_open', 'high':'bid_high', 'low':'bid_low', 'close':'bid_close'})
        for col in ['open', 'high', 'low', 'close']:
            df.loc[:,f'{col}'] = (df[f'bid_{col}'] + df[f'ask_{col}']) / 2
            
        
        return df
    else:
        return pd.DataFrame()
df = get_df_month('AUDUSD', 2020, 5, True)
df

,time,bid_open,bid_high,bid_low,bid_close,tick_volume,spread,date,ask_open,ask_high,ask_low,ask_close,open,high,low,close
0,1588284000,0.65206,0.65231,0.65203,0.65219,43,15,2020-04-30 22:00:00,0.65221,0.65246,0.65218,0.65234,0.652135,0.652385,0.652105,0.652265
1,1588284060,0.65219,0.65230,0.65219,0.65228,26,14,2020-04-30 22:01:00,0.65233,0.65244,0.65233,0.65242,0.652260,0.652370,0.652260,0.652350
2,1588284120,0.65229,0.65236,0.65228,0.65233,28,15,2020-04-30 22:02:00,0.65244,0.65251,0.65243,0.65248,0.652365,0.652435,0.652355,0.652405
3,1588284180,0.65233,0.65233,0.65223,0.65226,23,15,2020-04-30 22:03:00,0.65248,0.65248,0.65238,0.65241,0.652405,0.652405,0.652305,0.652335
4,1588284240,0.65226,0.65233,0.65221,0.65229,37,15,2020-04-30 22:04:00,0.65241,0.65248,0.65236,0.65244,0.652335,0.652405,0.652285,0.652365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30347,1590796500,0.66672,0.66683,0.66669,0.66679,27,18,2020-05-29 23:55:00,0.66690,0.66701,0.66687,0.66697,0.666810,0.666920,0.666780,0.666880
30348,1590796560,0.66681,0.66687,0.66652,0.66657,46,19,2020-05-29 23:56:00,0.66700,0.66706,0.66671,0.66676,0.666905,0.666965,0.666615,0.666665
30349,1590796620,0.66657,0.66675,0.66630,0.66632,58,55,2020-05-29 23:57:00,0.66712,0.66730,0.66685,0.66687,0.666845,0.667025,0.666575,0.666595
30350,1590796680,0.66632,0.66645,0.66618,0.66645,18,54,2020-05-29 23:58:00,0.66686,0.66699,0.66672,0.66699,0.666590,0.666720,0.666450,0.666720


## concatenate df

In [48]:
def concat_successive(df1,df2):
    end_time = df1.iloc[-1].time
    start_time = df2.iloc[0].time
    if end_time == start_time:
        return pd.concat([df1.iloc[:-1],df2],axis=0).reset_index(drop=True)
    else:
        return pd.concat([df1,df2],axis=0).reset_index(drop=True)

In [56]:
concat_successive(get_df_month(list_symbol[0], 2020, 6, True),get_df_month(list_symbol[0], 2020, 6+1, True))

,time,open,high,low,close,tick_volume,spread,date
0,1590969600,0.66565,0.66569,0.66551,0.66551,4,60,2020-06-01 00:00:00
1,1590969660,0.66567,0.66567,0.66567,0.66567,1,61,2020-06-01 00:01:00
2,1590969720,0.66567,0.66567,0.66554,0.66567,3,61,2020-06-01 00:02:00
3,1590969780,0.66567,0.66575,0.66567,0.66575,2,43,2020-06-01 00:03:00
4,1590969840,0.66561,0.66580,0.66561,0.66580,3,43,2020-06-01 00:04:00
...,...,...,...,...,...,...,...,...
64612,1596232560,0.71409,0.71409,0.71401,0.71409,31,14,2020-07-31 21:56:00
64613,1596232620,0.71409,0.71417,0.71408,0.71410,19,14,2020-07-31 21:57:00
64614,1596232680,0.71410,0.71443,0.71410,0.71442,38,14,2020-07-31 21:58:00
64615,1596232740,0.71443,0.71453,0.71433,0.71449,104,14,2020-07-31 21:59:00


## get list symbol

In [25]:
assetType = 'CFDs'
broker = 'VantageFX'
list_symbol = []
for symbol in mt5.symbols_get('*'):
    if symbol.path.split('\\')[0] == assetType:
        list_symbol.append(symbol.name)

## loop on symbols

In [26]:
def loop(list_symbol, assetType, broker, debug=False):
    if assetType not in os.listdir('D://Trading/Data'):
        os.mkdir(f'D://Trading/Data/{assetType}')
    if broker not in os.listdir(f'D://Trading/Data/{assetType}'):
        os.mkdir(f'D://Trading/Data/{assetType}/{broker}')
        os.mkdir(f'D://Trading/Data/{assetType}/{broker}/Price')
    for symbol in list_symbol:
        try:
            print(f'{datetime.datetime.now()} - importing data of {symbol}')
            mt5.symbol_select(symbol, True)
            keep_downloading = True
            for year in range(datetime.datetime.now().year,1999,-1):
                #print(f'{datetime.datetime.now()} - year {year}')
                for month in range(12,0,-1):                
                    if (year == datetime.datetime.now().year and month < datetime.datetime.now().month) or year < datetime.datetime.now().year:
                        if keep_downloading:

                            date = datetime.datetime(year, month, 1)
                            year_str = date_to_ymd_str(date)[0]
                            month_str = date_to_ymd_str(date)[1]
                            allowDL = symbol not in os.listdir(f'D://Trading/Data/{assetType}/{broker}/Price/')
                            if not allowDL:
                                list_year_str = os.listdir(f'D://Trading/Data/{assetType}/{broker}/Price/{symbol}/1min')
                                allowDL = year_str not in list_year_str
                                if not allowDL:                                
                                    filename = f'{year_str}-{month_str}.csv.gz'
                                    list_filename = os.listdir(f'D://Trading/Data/{assetType}/{broker}/Price/{symbol}/1min/{year_str}')
                                    allowDL = filename not in list_filename

                            if allowDL:

                                if debug:
                                    print(f'{datetime.datetime.now()} - year = {year}  month = {month}')

                                date = datetime.datetime(year, month, 1)
                                path = f'D://Trading/Data/{assetType}/{broker}/Price/:symbol/1min/:year/:year-:month.csv.gz'

                                params = {
                                    "symbol": symbol,
                                    "year": date_to_ymd_str(date)[0],
                                    "month": date_to_ymd_str(date)[1]
                                }
                                for key, item in params.items():
                                    path = path.replace(':'+key, item)

                                df = get_df_month(symbol, year, month, False)

                                if len(df) > 10 :       
                                    folders_to_create = path[4:].split('/')[:-1]
                                    tmp_path = 'D://'
                                    for folder in folders_to_create:
                                        if folder not in os.listdir(tmp_path):
                                            os.mkdir(tmp_path+folder)
                                        tmp_path += folder + '/'

                                    df.to_csv(path,index=False)
                                    
                                    update_infos(assetType, broker, 'Price', '1min', symbol, df)

                                else:
                                    print(f'{symbol} {year}-{month} is empty')
                                    keep_downloading = False
        except:
            print('error with',symbol)
                    
        
loop(list_symbol, assetType, broker, False)

2021-03-28 12:55:20.289605 - importing data of CHINA50
CHINA50 2015-5 is empty
2021-03-28 12:55:53.095719 - importing data of DAX30
DAX30 2008-12 is empty
2021-03-28 12:56:23.346665 - importing data of DJ30
DJ30 2008-9 is empty
2021-03-28 12:57:11.260295 - importing data of ES35
ES35 2018-10 is empty
2021-03-28 12:57:28.879268 - importing data of EU50
EU50 2009-3 is empty
2021-03-28 12:57:45.195238 - importing data of FRA40
FRA40 2017-7 is empty
2021-03-28 12:58:09.493060 - importing data of FTSE100
FTSE100 2008-9 is empty
2021-03-28 12:58:39.087925 - importing data of HSI
HSI 2015-5 is empty
2021-03-28 12:59:13.389406 - importing data of NAS100
NAS100 2012-10 is empty
2021-03-28 13:00:02.615117 - importing data of SA40
SA40 2020-12 is empty
2021-03-28 13:00:03.194694 - importing data of SP500
SP500 2009-8 is empty
2021-03-28 13:00:41.364912 - importing data of SPI200
SPI200 2007-4 is empty
2021-03-28 13:01:06.923519 - importing data of TW50
TW50 2020-6 is empty
2021-03-28 13:01:13.087